In [ ]:
import os, glob, pathlib
from loky import get_reusable_executor
import dill as pickle

In [ ]:
%run 'ER3BP Precession Dissipation.ipynb'

In [ ]:
qval = 4e-6

ps = {
    f1: f1val_outside,
    f2: f2val_outside,
    mu_p: qval,
    j: 2,
    Te: 1e3 * 2 * np.pi,
    Tm: 1e6 * 2 * np.pi,
    pom_p: np.pi,
}

H_eom = H_XYthTh

Xdot = plugin(H_eom.diff(Y) + Xdot_dis, ps)
Ydot = plugin(-H_eom.diff(X) + Ydot_dis, ps)
thdot = plugin(H_eom.diff(Th), ps)
Thdot = plugin(-H_eom.diff(th) + Thdot_dis, ps)
#display(Xdot, Ydot, thdot, Thdot)

Th_a = plugin(to_orbelts(coords_thTh[Th]), ps)
a_Th = plugin(to_orbelts(coords_thTh[L]), ps) ** 2
initVals = [0, 0, 0, Th_a.subs(a, 1.4)]

event1 = float(Th_a.subs(a, 1.2))
event2 = float(Th_a.subs(a, 2.))
tspan = (0, 2*np.pi * 1e6)
t_eval = np.linspace(tspan[0], tspan[1], 100000)


def f(t, Y):
    return Y[3] - event1


def fp(t, Y):
    return Y[3] - event2


f.terminal = True
fp.terminal = True

evs = [f, fp]

In [ ]:
epvals = np.linspace(0, 0.03, 20)
ompvals = np.zeros(61)
ompvals[:30] = -np.flip(np.logspace(-6, -2, 30))
ompvals[31:] = np.logspace(-6, -2, 30)
jobs = []
for i, epval in enumerate(epvals):
    for k, ompval in enumerate(ompvals):
        jobs = jobs + [(61*i+k, ompval, 0., epval, evs)]
print(len(jobs))
print(len(ompvals))
print(jobs)

In [ ]:
executors_solve_ivp = get_reusable_executor(max_workers=8)
results = list(executors_solve_ivp.map(solve_ivp, jobs))

In [ ]:
with open(f"/Users/jtlaune/wc-apsidal-precession/results-mup4e-6-er3bp-tpoutside-discapture.pkl", "wb") as f:
    pickle.dump(results,f)

In [ ]:
with open(f"/Users/jtlaune/wc-apsidal-precession/results-mup4e-6-er3bp-tpoutside-discapture.pkl", "rb") as f:
    results = pickle.load(f)

In [ ]:
with mpl.rc_context(analytic):
    fig, ax = plt.subplots(figsize=(6, 12))
fig.subplots_adjust(left=0.4)

trapit = 0
escit = 0
exit = 0

for result in results:
    evs = result[2]
    ps = result[1]
    trapped = True
    for ev in evs:
        if len(ev) > 0:
            trapped = False
    if trapped:
        if trapit == 0:
            ax.scatter(ps[3], ps[1], c="k", s=50, marker="s",label="Captured")
            trapit += 1
        else:
            ax.scatter(ps[3], ps[1], c="k", s=50, marker="s")
        #ax.text(ps[3], ps[1], ps[0], fontsize=10, c="c")
    else:
        if escit == 0:
            ax.scatter(ps[3], ps[1], c="r", s=50, marker="s",label="Escape")
            escit += 1
        else:
            ax.scatter(ps[3], ps[1], c="r", s=50, marker="s")
        #ax.text(ps[3], ps[1], ps[0], fontsize=10, c="m")

a00 = (3 / 2) ** (2.0 / 3)

eps = np.linspace(epvals[0], 0.03, 1000)
d1 = (
    1.5
    * 1.5
    * 4
    * np.sqrt(2 / 3 * eps * np.abs(f1val_outside) * qval)
    / 3 ** (2 / 3)
    / 2 ** (1 / 3)
)
d2 = (
    3
    / a00**0.5
    / 3 ** (2 / 3)
    * (8 / 3 * a00**2 * np.abs(f2val_outside) * qval) ** (2.0 / 3)
)
#ax.plot(eps, d1+d2, c="m", lw=6, ls="--")
ax.plot(eps, d1, c="m", lw=6, ls="--",label=r"$\delta_2$")
ax.axhline(y= d2, c="c", lw=6, ls="--",label=r"$\delta_2$")
ax.plot(eps, -d1, c="m", lw=6, ls="--" )
ax.axhline(y= -d2, c="c", lw=6, ls="--")

ax.set_xlabel(r"$e_p$")
ax.set_ylabel(r"$\omega_p/n_p$")
ax.legend(loc="upper left",fontsize=10,framealpha=1)

ax.xaxis.set_major_locator(ticker.MaxNLocator(4))

ax.set_ylim(-0.12, 0.12)
ax.set_yscale("symlog", linthresh=1e-6, linscale=0.3)
fig.tight_layout()

In [ ]:
sel = 814
#sel = 406
#sel = 394
for i, result in enumerate(results):
    ps = result[1]
    if ps[0] == sel:
        ind = i
        print("om_p=",ps[1])
        print("e_p=",ps[3])
print(ind)

sol = results[int(ind)][0]

mpl.rcParams["lines.markersize"] = 0.2
with mpl.rc_context(analytic):
    fig, ax = plt.subplots(5, figsize=(6, 8))
fig.subplots_adjust(left=0.2)
for axis in ax[2:]:
    axis.set_ylim(-np.pi, np.pi)
for axis in ax[:-1]:
    axis.xaxis.set_ticklabels([])
for axis in ax:
    axis.set_xlim((sol["t"][0], sol["t"][-1]))


#ax[0].twinx().scatter(sol["t"], (sol["a"])**(3/2),c="orange")
ax[0].scatter(sol["t"], sol["a"],c="k")
ax[1].scatter(sol["t"], sol["e"],c="k")
ax[2].scatter(sol["t"], sol["pom"],c="k")
th_0 = (sol["th"] - sol["pom"]) % (2 * np.pi)
th_0 = th_0 - 2 * np.pi * (th_0 > np.pi)
ax[3].scatter(sol["t"], th_0,c="k")
th_p = (sol["th"] ) % (2 * np.pi)
th_p = th_p - 2 * np.pi * (th_p > np.pi)
ax[4].scatter(sol["t"], th_p,c="k")

ax[0].set_ylabel(r"$a/a_p$")
ax[1].set_ylabel(r"$e$")
ax[2].set_ylabel(r"$\varpi$")
ax[3].set_ylabel(r"$\theta_0 - \varpi$")
ax[4].set_ylabel(r"$\theta_0 - \omega_p \tau$")

ax[4].ticklabel_format(axis="x",scilimits=(0,0))

ax[4].set_xlabel(r"$t/(2\pi/n_p)$")
fig.tight_layout()

In [ ]:
sel = 300
for i, result in enumerate(results):
    ps = result[1]
    if ps[0] == sel:
        ind = i
print(ind)

sol = results[int(ind)][0]
ps = results[int(ind)][1]

a0 = (3 / 2) ** (2.0 / 3)
print(a0)
Th_a.subs(a, a0)

H_plot = (
    H_eom.subs({X: X_gG, Y: Y_gG})
    .simplify()
    .subs(
        {
            om_p: ps[1],
            om: ps[2],
            e_p: ps[3],
            a_p: 1,
            f2: f2val_outside,
            mu_p: 3e-6,
            pom_p: 0,
            f1: 0,
            j: 2,
            Th: Th_a,
        }
    )
).evalf()

In [ ]:
x = symbols("x")
eq0 = H_plot.subs({G: 0.5 * sqrt(a) * e**2}).subs({e: (3e-6)**(1./3)}).subs(
    g, x - th
) - H_plot.subs(
    {
        a: a0,
        th: -g + pi / 2,
    }
)
eq = lambdify([x,a],eq0)

In [ ]:
sel = 756
#sel = 300
for i, result in enumerate(results):
    ps = result[1]
    if ps[0] == sel:
        ind = i
print(ind)


sol = results[int(ind)][0]
ps = results[int(ind)][1]
ompval = ps[1]
print(ompval)
indt1 = np.where(sol["t"] > 4.0004e5)[0][0]
indt2 = np.where(sol["t"] > 5e5)[0][0]
#indt2 = np.where(sol["t"] > 5e5)[0][0]
#indt3 = np.where(sol["t"] > 6e6)[0][0]

with mpl.rc_context(analytic):
    fig, ax = plt.subplots()
fig.subplots_adjust(left=0.2)

n = sol["a"][indt1:indt2]**(-3./2)
th2 = (sol["th"][indt1:indt2]-sol["pom"][indt1:indt2])%(2*np.pi)
th2 = th2 - 2*np.pi*(th2>np.pi)
ax.scatter(th2, n,s=10,c="k")#,c=sol["t"][indt1:indt2])

#print(np.average(sol["e"][indt3:]))
TH = np.linspace(-np.pi,np.pi,1000)
NN = np.linspace(.664,.669,1000)
TH,NN = np.meshgrid(TH,NN)
AA = NN**(-2./3)
cs = ax.contour(TH,NN,eq(TH,AA),levels=[-1.075e-7],colors="c")

ax.axhline(y=(2/(3))+ompval/3,c="m",ls="--")